# Data preparation
Contents:
- Add names. (done)
- Remove outliers - people who have probably incorrectly answered questions and people who have randomly answered questions. (done - check errors)
- Data preparation for Together Apart - deciding and selecting columns that are relevant for friend-finding. (done)
- Final cleaning - analysing and solving any final issues; exporting the final dataframe as a separate csv. (done)

## Add names
The Young People Survey dataset is anonymous. For our app, we will have the participants enter their name into the questionnaire. To make this dataset mimic the dataset we will create in our app, we need to give the participants names.

In [1]:
# import necessary packages
import pandas as pd
import numpy as np

In [2]:
# Read the  data and store as a Pandas dateframe
df = pd.read_csv("responses.csv")

In [3]:
df.describe()

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Shopping centres,Branded clothing,Entertainment spending,Spending on looks,Spending on gadgets,Spending on healthy eating,Age,Height,Weight,Number of siblings
count,1007.000000,1008.000000,1006.000000,1005.000000,1005.000000,1003.000000,1008.000000,1007.000000,1004.000000,1007.000000,...,1008.000000,1008.000000,1007.000000,1007.000000,1010.000000,1008.00000,1003.000000,990.000000,990.000000,1004.000000
mean,4.731877,3.328373,3.113320,2.288557,2.123383,2.956132,2.761905,3.471698,3.761952,2.361470,...,3.234127,3.050595,3.201589,3.106256,2.870297,3.55754,20.433699,173.514141,66.405051,1.297809
std,0.664049,0.833931,1.170568,1.138916,1.076136,1.252570,1.260845,1.161400,1.184861,1.372995,...,1.323062,1.306321,1.188947,1.205368,1.284970,1.09375,2.828840,10.024505,13.839561,1.013348
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,15.000000,62.000000,41.000000,0.000000
25%,5.000000,3.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,3.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,3.00000,19.000000,167.000000,55.000000,1.000000
50%,5.000000,3.000000,3.000000,2.000000,2.000000,3.000000,3.000000,4.000000,4.000000,2.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,4.00000,20.000000,173.000000,64.000000,1.000000
75%,5.000000,4.000000,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,5.000000,3.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.00000,22.000000,180.000000,75.000000,2.000000
max,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,30.000000,203.000000,165.000000,10.000000


In [4]:
# column labels
df.columns

Index(['Music', 'Slow songs or fast songs', 'Dance', 'Folk', 'Country',
       'Classical music', 'Musical', 'Pop', 'Rock', 'Metal or Hardrock',
       ...
       'Age', 'Height', 'Weight', 'Number of siblings', 'Gender',
       'Left - right handed', 'Education', 'Only child', 'Village - town',
       'House - block of flats'],
      dtype='object', length=150)

In [5]:
# Shape of data
df.shape

(1010, 150)

We need to know which participants are male and which are female, so that we can add names of the correct gender.

In [6]:
# number of males, females, and unknown gender
n_male = (df['Gender'] == 'male').sum()
n_female = (df['Gender'] == 'female').sum()
n_unknown = df['Gender'].isnull().sum()
print(n_male, n_female, n_unknown)

411 593 6


There are 6 participants who did not give their gender, we'll give these people male names as there are fewer males in the dataset.

We've created a list of male names and a list of female names which we can import and clean up as follows.

In [7]:
# Import name lists
df_female_names = pd.read_csv('female_names.csv')[:n_female]
df_male_names = pd.read_csv('male_names.csv')[:n_male + n_unknown]

# remove non-ascii characters which have occurred because the names were copied from a website.
df_male_names['Name'] = df_male_names['Name'].apply(lambda x: x.replace('\xa0', ' '))
df_female_names['Name'] = df_female_names['Name'].apply(lambda x: x.replace('\xa0', ' '))

In [8]:
# Add new column to main dataframe for name
df['Name'] = ''

In [9]:
# Set the indices of the male name dataframe to be the indices of the males (and unknowns) in the
# main dataframe.
df_male_names = df_male_names.set_index(df.index[(df['Gender'] == 'male') | df['Gender'].isnull()])

# Do the same for females.
df_female_names = df_female_names.set_index(df.index[(df['Gender'] == 'female')])

In [10]:
# Add names into main dataframe.
df['Name'] = df_male_names
df.loc[df['Gender'] == 'female', 'Name'] = df_female_names

Now we have a column of names with the appropriate genders.

In [11]:
# Print the gender and names of the last few rows.
df[['Gender', 'Name']].tail()

,Gender,Name
1005,female,Caroline Wilks
1006,male,Roy Martin
1007,female,Lelia Williams
1008,female,Lauren Williamson
1009,male,Ciaran May


## Remove outliers

Explore data for abnormal values. Only the Age, Height, Weight, and Number of siblings questions allowed the participant to enter any value. So let's check the extreme values of these columns to see if there are any anomolies.

In [12]:
df[['Age', 'Height', 'Weight', 'Number of siblings']].agg(['min', 'max'])

,Age,Height,Weight,Number of siblings
min,15.0,62.0,41.0,0.0
max,30.0,203.0,165.0,10.0


Age and number of siblings look ok. The height is measured in cm and the weight in kg, so the min height and max weight look like errors. Let's look more closely at these values.

In [13]:
df[df['Height'] < 120]

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats,Name
676,5.0,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,...,62.0,55.0,2.0,female,right handed,college/bachelor degree,no,city,house/bungalow,Sarah Baker


That 62cm height looks like an error, given that she weighs 55kg. Let's replace it with NaN.

In [14]:
df.loc[676,'Height'] = np.NaN

In [15]:
df[df['Weight'] > 130]

,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats,Name
885,3.0,4.0,3.0,2.0,2.0,2.0,3.0,4.0,4.0,4.0,...,NaN,165.0,0.0,female,right handed,secondary school,yes,city,house/bungalow,Keava Mone
992,4.0,4.0,4.0,1.0,4.0,4.0,1.0,3.0,4.0,4.0,...,200.0,150.0,1.0,male,right handed,masters degree,no,city,block of flats,Stacey Lewis


There are very few people with a weight of 150kg or more. The participants may have misread the units, thinking they were entering 150lb instead. Let's replace these extreme values with NaN

In [16]:
df.loc[885,'Weight'] = np.NaN
df.loc[992,'Weight'] = np.NaN

Next, let's check whether there are any people who have incorrectly answered whether they are an only child and how many siblings they have. If someone is an only child, they have no siblings.

In [17]:
df[['Only child', 'Number of siblings']][(df['Only child'] == 'yes') & (df['Number of siblings'] >= 1)]

,Only child,Number of siblings
3,yes,1.0
25,yes,1.0
28,yes,2.0
47,yes,1.0
48,yes,1.0
...,...,...
956,yes,2.0
964,yes,3.0
975,yes,1.0
976,yes,1.0


So there are 95 people who either incorrectly answered whether they are an only child or the number of siblings they have. Let's change these values to NaN for these participants.

In [18]:
i_error = df[['Only child', 'Number of siblings']][(df['Only child'] == 'yes') & (df['Number of siblings'] >= 1)].index
df.loc[i_error, ['Only child', 'Number of siblings']] = np.NaN

## Data preparation for Together Apart
In this section, we decide on the columns that are relevant for friend-finding. We also get the data ready to match with the Together Apart registration form/questionaire.

### Viewing and analysing the data

In [19]:
# Reading the columns

print(df.columns.values)

['Music' 'Slow songs or fast songs' 'Dance' 'Folk' 'Country'
 'Classical music' 'Musical' 'Pop' 'Rock' 'Metal or Hardrock' 'Punk'
 'Hiphop, Rap' 'Reggae, Ska' 'Swing, Jazz' 'Rock n roll' 'Alternative'
 'Latino' 'Techno, Trance' 'Opera' 'Movies' 'Horror' 'Thriller' 'Comedy'
 'Romantic' 'Sci-fi' 'War' 'Fantasy/Fairy tales' 'Animated' 'Documentary'
 'Western' 'Action' 'History' 'Psychology' 'Politics' 'Mathematics'
 'Physics' 'Internet' 'PC' 'Economy Management' 'Biology' 'Chemistry'
 'Reading' 'Geography' 'Foreign languages' 'Medicine' 'Law' 'Cars'
 'Art exhibitions' 'Religion' 'Countryside, outdoors' 'Dancing'
 'Musical instruments' 'Writing' 'Passive sport' 'Active sport'
 'Gardening' 'Celebrities' 'Shopping' 'Science and technology' 'Theatre'
 'Fun with friends' 'Adrenaline sports' 'Pets' 'Flying' 'Storm' 'Darkness'
 'Heights' 'Spiders' 'Snakes' 'Rats' 'Ageing' 'Dangerous dogs'
 'Fear of public speaking' 'Smoking' 'Alcohol' 'Healthy eating'
 'Daily events' 'Prioritising workload' 'Wri

In [20]:
# Checking the shape of the dataset
df.shape

(1010, 151)

In [21]:
# Checking the mean value, type, and length of some columns (overwriten)
df['Music'].mean

<bound method Series.mean of 0       5.0
1       4.0
2       5.0
3       5.0
4       5.0
       ... 
1005    5.0
1006    4.0
1007    4.0
1008    5.0
1009    5.0
Name: Music, Length: 1010, dtype: float64>

## Deciding on the columns we'll use

For the purposes of this project we decided to continue with the following columns (please see the lists below).

In the registration form they will be grouped in two categories: "Activities" and "Interesting Subjects".

The form will provide a 1 to 5 linkert scale for each subject (which is a column name in this dataset), where 1 means "not interested" and 5 - "very interested". The user will be able to choose the level of their interest, to then match up with someone, based on similarities of what they would like to do with their new buddy.


#### Activities (Let's Do This - Together Apart) (I am looking for an activity buddy.)
* Dancing
* Singing ("Musical instruments" in this dataset)
* Writing
* Meditation ("Passive sport" in this dataset)
* Playing games ("Fun with friends" in this dataset)
* Active sports (such as yoga; "Active sport" in this dataset)
* Being creative ("Art exhibition" in this dataset)
* Acting ("Theatre" in this dataset)
* Cooking ("Healthy eating" in this dataset)
* Gardening
* Pets

#### Interesting Subjects (Let's Talk - Together Apart) (I would like to talk about this with a buddy.)
* Music
* Movies
* Reading
* Foreign languages
* Daily events
* Celebrities
* Science and technology
* Future goals ("Thinking ahead" in this dataset)
* Sharing my past ("Changing the past" in this dataset)
* Dreams
* Loneliness
* Health
* Mental wellbeing ("Mood swings" in this dataset)
* Life struggles

### Renamed columns

In [22]:
# Implementing the decision above by altering the column names
# and saving those changes in a new dataframe: df_col_renamed.

df_col_renamed = df.rename(columns={'Musical instruments': 'Singing',
                           'Passive sport': 'Meditation',
                           'Fun with friends': 'Playing games',
                           'Active sport': 'Active sports',
                           'Art exhibitions': 'Being creative',
                           'Theatre': 'Acting',
                           'Healthy eating': 'Cooking',
                           'Thinking ahead': 'Future goals',
                           'Changing the past': 'Sharing my past',
                           'Mood swings': 'Mental wellbeing'})

print(df_col_renamed.columns.values)

['Music' 'Slow songs or fast songs' 'Dance' 'Folk' 'Country'
 'Classical music' 'Musical' 'Pop' 'Rock' 'Metal or Hardrock' 'Punk'
 'Hiphop, Rap' 'Reggae, Ska' 'Swing, Jazz' 'Rock n roll' 'Alternative'
 'Latino' 'Techno, Trance' 'Opera' 'Movies' 'Horror' 'Thriller' 'Comedy'
 'Romantic' 'Sci-fi' 'War' 'Fantasy/Fairy tales' 'Animated' 'Documentary'
 'Western' 'Action' 'History' 'Psychology' 'Politics' 'Mathematics'
 'Physics' 'Internet' 'PC' 'Economy Management' 'Biology' 'Chemistry'
 'Reading' 'Geography' 'Foreign languages' 'Medicine' 'Law' 'Cars'
 'Being creative' 'Religion' 'Countryside, outdoors' 'Dancing' 'Singing'
 'Writing' 'Meditation' 'Active sports' 'Gardening' 'Celebrities'
 'Shopping' 'Science and technology' 'Acting' 'Playing games'
 'Adrenaline sports' 'Pets' 'Flying' 'Storm' 'Darkness' 'Heights'
 'Spiders' 'Snakes' 'Rats' 'Ageing' 'Dangerous dogs'
 'Fear of public speaking' 'Smoking' 'Alcohol' 'Cooking' 'Daily events'
 'Prioritising workload' 'Writing notes' 'Workaholism' 

### Final shortened dataframe

In [23]:
# Here I wanted to drop the unused columns and save the final vs into a new file
# but soon realised it would be much faster to just create a new df with listed columns (ta - for together apart :D) 

df_ta = df_col_renamed[['Name', 'Dancing', 'Singing', 'Writing', 'Meditation',
                        'Playing games', 'Active sports', 'Being creative',
                        'Acting', 'Cooking', 'Gardening', 'Pets', 'Music',
                        'Movies', 'Reading', 'Foreign languages', 'Daily events',
                        'Celebrities', 'Science and technology', 'Future goals',
                        'Sharing my past', 'Dreams', 'Loneliness', 'Health', 'Mental wellbeing', 'Life struggles']]

print(df_ta.columns.values)

['Name' 'Dancing' 'Singing' 'Writing' 'Meditation' 'Playing games'
 'Active sports' 'Being creative' 'Acting' 'Cooking' 'Gardening' 'Pets'
 'Music' 'Movies' 'Reading' 'Foreign languages' 'Daily events'
 'Celebrities' 'Science and technology' 'Future goals' 'Sharing my past'
 'Dreams' 'Loneliness' 'Health' 'Mental wellbeing' 'Life struggles']


## Final cleaning

In [24]:
#Looking at the full final dataframe - uncomment to prints

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# print(df_ta)

From looking at this dataframe, we can notice some issues that need to be solved:

* "Dreams" column is of a different type
* The dataset contains missing values 
* "Name" column has trailing

### Type casting

In [25]:
# Checking the types of final columns

print(df_ta.dtypes)

Name                       object
Dancing                   float64
Singing                   float64
Writing                   float64
Meditation                float64
Playing games             float64
Active sports             float64
Being creative            float64
Acting                    float64
Cooking                   float64
Gardening                 float64
Pets                      float64
Music                     float64
Movies                    float64
Reading                   float64
Foreign languages         float64
Daily events              float64
Celebrities               float64
Science and technology    float64
Future goals              float64
Sharing my past           float64
Dreams                      int64
Loneliness                float64
Health                    float64
Mental wellbeing          float64
Life struggles            float64
dtype: object


In [26]:
# Changing the Dreams column to be of float type.

df_ta = df_ta.astype({'Dreams': 'float64'})
print(df_ta.dtypes)

Name                       object
Dancing                   float64
Singing                   float64
Writing                   float64
Meditation                float64
Playing games             float64
Active sports             float64
Being creative            float64
Acting                    float64
Cooking                   float64
Gardening                 float64
Pets                      float64
Music                     float64
Movies                    float64
Reading                   float64
Foreign languages         float64
Daily events              float64
Celebrities               float64
Science and technology    float64
Future goals              float64
Sharing my past           float64
Dreams                    float64
Loneliness                float64
Health                    float64
Mental wellbeing          float64
Life struggles            float64
dtype: object


### Imputation of missing values

In [27]:
# Checking the number of NaNs
df_ta.isnull().sum().sum()

110

In [28]:
# Replacing NaNs, with the most frequent value of the columns (axis 0) that contain the missing values:
df_ta = df_ta.apply(lambda x:x.fillna(x.value_counts().index[0]))

# Checking the number of NaNs after the change
df_ta.isnull().sum().sum()

0

### Trailing removal

In [29]:
# Cleaning up the whitespace in the "Name" column

df_ta['Name'] = df_ta['Name'].apply(str.strip)

## Final Checks

In [30]:
# Looking at the full final dataframe

print(df_ta)

                   Name  Dancing  Singing  Writing  Meditation  Playing games  \
0         Sally Abraham      3.0      3.0      2.0         1.0            5.0   
1       Rebecca Acheson      1.0      1.0      1.0         1.0            4.0   
2        Carole Ackland      5.0      5.0      5.0         5.0            5.0   
3       Hayley Alderton      1.0      1.0      3.0         1.0            2.0   
4        Nicola Allison      1.0      3.0      1.0         3.0            4.0   
...                 ...      ...      ...      ...         ...            ...   
1005     Caroline Wilks      2.0      3.0      4.0         4.0            5.0   
1006         Roy Martin      5.0      5.0      1.0         1.0            5.0   
1007     Lelia Williams      1.0      2.0      1.0         2.0            3.0   
1008  Lauren Williamson      4.0      1.0      4.0         2.0            5.0   
1009         Ciaran May      2.0      1.0      2.0         5.0            5.0   

      Active sports  Being 

In [31]:
df_ta.describe()

,Dancing,Singing,Writing,Meditation,Playing games,Active sports,Being creative,Acting,Cooking,Gardening,...,Daily events,Celebrities,Science and technology,Future goals,Sharing my past,Dreams,Loneliness,Health,Mental wellbeing,Life struggles
count,1010.000000,1010.000000,1010.00000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,...,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000,1010.000000
mean,2.457426,2.322772,1.89604,3.411881,4.559406,3.298020,2.580198,3.024752,3.031683,1.900990,...,3.074257,2.359406,3.232673,3.412871,2.952475,3.297030,2.887129,3.250495,3.257426,3.031683
std,1.450426,1.512674,1.28577,1.408124,0.736245,1.504962,1.323720,1.320112,0.935473,1.173755,...,1.114563,1.270437,1.278906,1.135442,1.277122,0.683148,1.131201,1.074815,1.042729,1.372600
min,1.000000,1.000000,1.00000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.00000,2.000000,4.000000,2.000000,1.000000,2.000000,3.000000,1.000000,...,2.000000,1.000000,2.000000,3.000000,2.000000,3.000000,2.000000,3.000000,3.000000,2.000000
50%,2.000000,2.000000,1.00000,4.000000,5.000000,3.000000,2.000000,3.000000,3.000000,1.000000,...,3.000000,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,3.00000,5.000000,5.000000,5.000000,4.000000,4.000000,4.000000,3.000000,...,4.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [32]:
# Checking the shape of the dataset
df_ta.shape

(1010, 26)

### All looks good!

In [38]:
# Merging the final cleaned dataframe ("df_ta") with the "df" for easier use in future
df = df_ta

df

,Name,Dancing,Singing,Writing,Meditation,Playing games,Active sports,Being creative,Acting,Cooking,...,Daily events,Celebrities,Science and technology,Future goals,Sharing my past,Dreams,Loneliness,Health,Mental wellbeing,Life struggles
0,Sally Abraham,3.0,3.0,2.0,1.0,5.0,5.0,1.0,2.0,4.0,...,2.0,1.0,4.0,2.0,1.0,4.0,3.0,1.0,3.0,1.0
1,Rebecca Acheson,1.0,1.0,1.0,1.0,4.0,1.0,2.0,2.0,3.0,...,3.0,2.0,3.0,4.0,4.0,3.0,2.0,4.0,4.0,1.0
2,Carole Ackland,5.0,5.0,5.0,5.0,5.0,2.0,5.0,5.0,3.0,...,1.0,1.0,2.0,5.0,5.0,1.0,5.0,2.0,4.0,4.0
3,Hayley Alderton,1.0,1.0,3.0,1.0,2.0,1.0,5.0,1.0,3.0,...,4.0,2.0,3.0,3.0,5.0,3.0,5.0,1.0,5.0,3.0
4,Nicola Allison,1.0,3.0,1.0,3.0,4.0,1.0,1.0,2.0,4.0,...,3.0,3.0,3.0,5.0,4.0,3.0,3.0,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,Caroline Wilks,2.0,3.0,4.0,4.0,5.0,4.0,5.0,5.0,3.0,...,3.0,4.0,3.0,2.0,1.0,3.0,4.0,4.0,3.0,4.0
1006,Roy Martin,5.0,5.0,1.0,1.0,5.0,5.0,1.0,1.0,3.0,...,1.0,1.0,5.0,5.0,4.0,3.0,1.0,3.0,1.0,1.0
1007,Lelia Williams,1.0,2.0,1.0,2.0,3.0,2.0,1.0,2.0,2.0,...,3.0,1.0,3.0,4.0,5.0,3.0,4.0,3.0,2.0,5.0
1008,Lauren Williamson,4.0,1.0,4.0,2.0,5.0,4.0,5.0,5.0,3.0,...,3.0,3.0,5.0,3.0,3.0,3.0,3.0,4.0,4.0,5.0


In [36]:
# Exporting the final version of the dataframe as a .csv file
# (commented out so it won't save on another run automatically)
# df.to_csv('TA_PreData_noNaNs.csv')

### Notes

We have 25 subjects that will be used in the registration form and 1010 entries for each that we can already work from to normalise the scores, implement machine learning algoritm to match users and create data visualisation for us and the user.